In [1]:
import librosa
import h5py
import numpy as np
import sklearn
from sklearn.preprocessing import normalize

import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation

Using TensorFlow backend.


In [2]:
file_list_bed = librosa.util.find_files('data/bed', ext = 'wav')
file_list_bed = list(set(file_list_bed)) # 겹치는 파일을 집합으로 만들어서 중복없애기
file_list_bird = librosa.util.find_files('data/bird', ext = 'wav')
file_list_bird = list(set(file_list_bird))
file_list_happy = librosa.util.find_files('data/happy', ext = 'wav')
file_list_happy = list(set(file_list_happy))

In [ ]:
for file in file_list_bed:
    y, sr = librosa.load(file)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    name = file[-22:-4]
    
    # reshape
    mfcc = mfcc.reshape(1,-1)
    #print(mfcc.shape)
    
    # normalization
    mfcc = normalize(mfcc)
    #print(mfcc.shape)
    
    # write
    if(mfcc.shape != (1,880)):
        h5f = h5py.File("no_feature/bed" + name + ".h5", 'w')
    else:
        h5f = h5py.File("feature/bed" + name + ".h5", 'w')
        h5f.create_dataset('arr', data=mfcc)
        h5f.close()
print(mfcc.shape)

In [4]:
for file in file_list_bird:
    y, sr = librosa.load(file)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    name = file[-22:-4]
    
    # reshape
    mfcc = mfcc.reshape(1,-1)
    #print(mfcc.shape)
    
    # normalization
    mfcc = normalize(mfcc)
    #print(mfcc.shape)
    
    # write
    if(mfcc.shape != (1,880)):
        h5f = h5py.File("no_feature/bird" + name + ".h5", 'w')
    else:
        h5f = h5py.File("feature/bird" + name + ".h5", 'w')
        h5f.create_dataset('arr', data=mfcc)
        h5f.close()

In [5]:
for file in file_list_happy:
    y, sr = librosa.load(file)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    name = file[-22:-4]
    
    # reshape
    mfcc = mfcc.reshape(1,-1)
    #print(mfcc.shape)
    
    # normalization
    mfcc = normalize(mfcc)
    #print(mfcc.shape)

    # write
    if(mfcc.shape != (1,880)):
        h5f = h5py.File("no_feature/happy" + name + ".h5", 'w')
    else:
        h5f = h5py.File("feature/happy" + name + ".h5", 'w')
        h5f.create_dataset('arr', data=mfcc)
        h5f.close()

In [6]:
h5_list_bed = librosa.util.find_files('feature/bed', ext = 'h5') # 읽기전용 리스트
h5_list_bed = list(set(h5_list_bed)) # 겹치는 파일을 집합으로 만들어서 중복없애기
#h5_list_bird = librosa.util.find_files('feature/bird', ext = 'h5')
#h5_list_bird = list(set(h5_list_bird))
#h5_list_happy = librosa.util.find_files('feature/happy', ext = 'h5')
#h5_list_happy = list(set(h5_list_happy))

In [7]:
arr_bed = np.empty((0,880)) # 빈 배열 생성
y_bed = np.empty((0,3)) # label 배열
for file in h5_list_bed:
    name = file[-21:-3]
    h5f = h5py.File("feature/bed" + name + ".h5", 'r')
    b = h5f['arr'][:]
    k = np.array(b) # 배열에 h5파일 넣기
    arr_bed = np.vstack((arr_bed,k)) # h5배열 쌓기
    h5f.close()
    arr_label = np.array([1,0,0])
    y_bed = np.vstack((y_bed,arr_label))
print(y_bed.shape)
print(y_bed)#test

(1484, 3)


In [8]:
arr_bird = np.empty((0,880)) # 빈 배열 생성
y_bird = np.empty((0,3)) # label 배열
for file in h5_list_bird:
    name = file[-21:-3]
    h5f = h5py.File("feature/bird" + name + ".h5", 'r')
    b = h5f['arr'][:]
    k = np.array(b) # 배열에 h5파일 넣기
    arr_bird = np.vstack((arr_bird,k)) # h5배열 쌓기
    h5f.close()
    arr_label = np.array([0,1,0])
    y_bird = np.vstack((y_bird,arr_label))
print(y_bird.shape)

(1521, 3)


In [9]:
arr_happy = np.empty((0,880)) # 빈 배열 생성
y_happy = np.empty((0,3)) # label 배열
for file in h5_list_happy:
    name = file[-21:-3]
    h5f = h5py.File("feature/happy" + name + ".h5", 'r')
    b = h5f['arr'][:]
    k = np.array(b) # 배열에 h5파일 넣기
    arr_happy = np.vstack((arr_happy,k)) # h5배열 쌓기
    h5f.close()
    arr_label = np.array([0,0,1])
    y_happy = np.vstack((y_happy,arr_label))
print(y_happy.shape)

(1549, 3)


In [10]:
x = np.vstack((arr_bed,arr_bird,arr_happy)) # bed, happy의 추출한 값을 합치기
#x = np.vstack((x,arr_happy))
y = np.vstack((y_bed,y_bird,y_happy)) # bed, bird의 라벨 합치기
print(x.shape) # 데이터중에 쓸 수 있는 값의 갯수
print(y.shape)

(4554, 880)
(4554, 3)


In [11]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print(x_train.shape)
print(x_test.shape)

(3643, 880)
(911, 880)


In [12]:
model = Sequential()
model.add(Dense(units=64, kernel_initializer='random_uniform', input_dim=880, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

In [13]:
adam = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

In [14]:
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy']) # optimizer -> lr조절
#adam vs sgd

In [15]:
hist = model.fit(x_train, y_train, epochs=200, batch_size=64) # epochs -> 전체 데이터를 학습하는 횟수 / batch_size -> 데이터를 나누는 수
# ex) data 10000개, epochs = 20, batch_size = 1000일 때 데이터 10000개를 1000개씩 나눠서 10번 돌고 총 데이터를 20번 학습 

Epoch 1/200
3643/3643 [==============================] - 0s - loss: 0.7907 - acc: 0.6923     
Epoch 2/200
3643/3643 [==============================] - 0s - loss: 0.4666 - acc: 0.8293     
Epoch 3/200
3643/3643 [==============================] - 0s - loss: 0.4143 - acc: 0.8411     
Epoch 4/200
3643/3643 [==============================] - 0s - loss: 0.3866 - acc: 0.8540     
Epoch 5/200
3643/3643 [==============================] - 0s - loss: 0.3794 - acc: 0.8548     
Epoch 6/200
3643/3643 [==============================] - 0s - loss: 0.3507 - acc: 0.8715     
Epoch 7/200
3643/3643 [==============================] - 0s - loss: 0.3369 - acc: 0.8740     
Epoch 8/200
3643/3643 [==============================] - 0s - loss: 0.3233 - acc: 0.8784     
Epoch 9/200
3643/3643 [==============================] - 0s - loss: 0.2970 - acc: 0.8916     
Epoch 10/200
3643/3643 [==============================] - 0s - loss: 0.2741 - acc: 0.9001     
Epoch 11/200
3643/3643 [==============================] - 0

In [16]:
# 모델 평가하기
loss_and_metrics = model.evaluate(x_test, y_test, batch_size=32)
print(loss_and_metrics)

911/911 [==============================] - 0s     
[0.52014137154729256, 0.91986827661909987]
